In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

***EXERCISE 1***

1. Implementation of PCA.

In [ ]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X):
        #we prepare the data by standardizing it
        self.mean = np.mean(X, axis=0)
        X = X - self.mean

        #we create the covariance matrix which is need for the eigenvalue decomp
        cov_matrix = np.cov(X.T)

        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        #take the principal components
        eigenvectors = eigenvectors.T
        idxs = np.argsort(eigenvalues)[::-1]
        self.components = eigenvectors[idxs][:self.n_components]

    def transform(self, X):
        #dot product the data and the principal components
        X = X - self.mean
        return np.dot(X, self.components.T)